In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path="../.env")

open_api_key = os.getenv("open_api_key")

openAI_params = {
    'api_key': open_api_key
}
client = OpenAI(**openAI_params)

#### Conversation State
It is important for preserving information across multiple messages or turns in a conversation.

#### 1. Manually Manage conversation state

In [19]:
history = [
    {
        "role": "user",
        "content": "tell me a joke"
    }
]

# Generate a response
response1 = client.responses.create( # Notice the difference in responses.create and complettion.create
    model="gpt-4o-mini",
    input=history,
    store=True
)

# Append the response to history
history += [{'role': res.role, "content": res.content} for res in response1.output]


# Add a new question
history += [{'role': 'user', 'content': "Tell me a another joke"}]


response2 = client.responses.create( # Notice the difference in responses.create and complettion.create
    model="gpt-4o-mini",
    input=history,
    store=True
)

response2.output_text

'Why don’t skeletons fight each other?\n\nThey don’t have the guts!'

### 2. OpenAI APIs for conversation state
Share context across generated responses with the `previous_response_id` parameter. This parameter lets you chain responses and create a threaded conversation.

Note: *Even when using previous_response_id, all previous input tokens for responses in the chain are billed as input tokens in the API.*

In [22]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="tell me a joke",
)
print(response.output_text)

second_response = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response.id, # Provide the previous context to the model
    input=[{"role": "user", "content": "explain why this is funny."}],
)
print(second_response.output_text)

Why did the scarecrow win an award?

Because he was outstanding in his field!
The joke is a pun that plays on the dual meaning of "outstanding." 

1. **Literal meaning**: A scarecrow is physically "standing out" in a field, as it's placed there to scare away birds.
2. **Figurative meaning**: "Outstanding" can also mean exceptional or excellent at something.

Combining these meanings creates a humorous twist. The unexpected connection between the scarecrow's role and the notion of receiving an award adds to the joke's charm. Puns often rely on wordplay like this, which can elicit a groan or a chuckle!


#### Managing the context window
Understanding context windows will help you successfully create threaded conversations and manage state across model interactions.

**Context Window**: Maximum number of tokens that can be used in a single request.

As your inputs become more complex, or you include more turns in a conversation, you'll need to consider both output token and context window limits.